In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT, OMEROWEB
from omero.gateway import BlitzGateway
import matplotlib.pyplot as plt
import ezomero

In [ ]:
conn = BlitzGateway(OMEROUSER, OMEROPASS, port=OMEROPORT, host=OMEROHOST)
ret = conn.connect()
print(ret)

In [ ]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

# Check if you are an Administrator
print("   Is Admin:", conn.isAdmin())
if not conn.isFullAdmin():
    # If 'Restricted Administrator' show privileges
    print(conn.getCurrentAdminPrivileges())

print("Member of:")
for g in conn.getGroupsMemberOf():
    print("   ID:", g.getName(), " Name:", g.getId())
group = conn.getGroupFromContext()
print("Current group: ", group.getName())

# List the group owners and other members
owners, members = group.groupSummary()
print("   Group owners:")
for o in owners:
    print("     ID: %s %s Name: %s" % (
        o.getId(), o.getOmeName(), o.getFullName()))

print("Owner of:")
for g in conn.listOwnedGroups():
    print("   ID: ", g.getName(), " Name:", g.getId())


In [ ]:
image_id = 5980
img, pix = ezomero.get_image(conn, image_id, xyzct=True)
print(pix.shape)
plt.imshow(pix[:,:,0,0,0].T)

In [ ]:
dataset_id = 859
id_list = ezomero.get_image_ids(conn, dataset_id)
print(id_list)

In [ ]:
image_id = 6028
ann_ids = ezomero.get_map_annotation_ids(conn, 'Image', image_id)
print(ann_ids)

In [ ]:
ezomero.print_map_annotation(conn, ann_ids[0])

In [ ]:
group_id = ezomero.get_group_id(conn,'training')
print(group_id)

In [ ]:
user_id = ezomero.get_user_id(conn, 'trainer-1')
print(user_id)

In [ ]:
ezomero.print_projects(conn)

In [ ]:
ezomero.print_datasets(conn,557)

In [ ]:
proj_id = 557
data_id = ezomero.post_dataset(conn, 'test_dataset',proj_id)

In [ ]:
from numpy.random import rand
img = rand(1000,1000,1,1,1)
img_id = ezomero.post_image(conn, img, 'randomimage.tif', dataset_id=data_id)

In [ ]:
ann_id = ezomero.post_map_annotation(conn, 'Image', img_id, {"test": 1, "test2": 2}, 'test_ns')